# 01 - clientes novos por regiao por periodo r


In [ ]:
''' mes / centro oeste / ...... / sul / total
total / ..... /  '''

In [ ]:


# -- 1 - definindo dfs e colunas
df_clientes = df_clientes_masquerade[['Cd Uf','Nm Cidade','Dt Cadastro']]
df_estados_regiões = pd.DataFrame(estados_regiões)
df_novos = ['regiao', 'Nm cidade', 'Quant']

# -- 2 - definindo a região 
df_clientes = df_clientes.merge(df_estados_regiões[['Cd Uf', 'Região']],
                                on='Cd Uf', how='left' )

# -- 3 definindo periodo 
anos_meses = {2024: [1,2,3,4,5,6,8,9,10,11,12],
        2025: [1,2,3,4,5,6]}

# -- 4 - filtrando pelo peridodo
dados = [] # paera pord4er atribuir tudo depois 

for ano, meses in anos_meses.items():
    for mes in meses:
        primeiro_dia = datetime(ano, mes, 1)
        ultimo_dia = (primeiro_dia + relativedelta(months=1)) - relativedelta(days=1)
        
        df_novos_raw = df_clientes[
            (df_clientes['Dt Cadastro'] >= primeiro_dia) & 
            (df_clientes['Dt Cadastro'] <= ultimo_dia)
        ]

        for index, row in df_novos_raw.iterrows():
            dados.append({
                "Ano": ano,
                "Mês": mes,
                "Nm Cidade" : row['Nm Cidade'],
                "Região": row["Região"]
                 })

df_novos_raw = pd.DataFrame(dados)
df_novos_raw['Periodo'] = df_novos_raw['Ano'].astype(str) + '-' + df_novos_raw['Mês'].astype(str).str.zfill(2)

df_novos_2024 = df_novos_raw[df_novos_raw['Ano'] == 2024]
df_novos_2024.drop(columns=['Ano','Mês'], inplace=True)
df_novos_2024.groupby('Periodo')['Região'].count() .sum()

In [ ]:
# === 1. Leitura dos dados ===
df_sonia_emanu = pd.read_excel('emauel_sonia.xlsx')
# === 2. Garantir que a coluna de data está no formato datetime ===
df_nfs_masquerade['Dt. emissão'] = pd.to_datetime(df_nfs_masquerade['Dt. emissão'], errors='coerce')

# === 3. Definir a data limite para os "últimos 3 meses" (ex: a partir de fevereiro/2025) ===
data_limite = pd.Timestamp('2025-02-01')

# === 4. Identificar CPFs com nota fiscal emitida após a data limite ===
clientes_com_nf_recent = df_nfs_masquerade[
    df_nfs_masquerade['Dt. emissão'] >= data_limite
]['Clie Cgc Cpf'].unique()

# === 5. Filtrar df_sonia_emanu: manter apenas quem NÃO comprou após a data limite ===
df_sonia_emanu_sem_nf_recent = df_sonia_emanu[
    ~df_sonia_emanu['Cgc Cpf'].isin(clientes_com_nf_recent)
]

# === 6. Obter a data da última compra de cada cliente (mesmo que antiga) ===
ultima_compra = df_nfs_masquerade.groupby('Clie Cgc Cpf')['Dt. emissão'].max().reset_index()
ultima_compra.rename(columns={'Dt. emissão': 'Dt. última compra'}, inplace=True)

# === 7. Juntar a última compra no DataFrame filtrado ===
df_sonia_emanu_sem_nf_recent = df_sonia_emanu_sem_nf_recent.merge(
    ultima_compra,
    how='left',
    left_on='Cgc Cpf',
    right_on='Clie Cgc Cpf'
)

# === 8. Limpar colunas duplicadas do merge (opcional) ===
df_sonia_emanu_sem_nf_recent.drop(columns=['Clie Cgc Cpf'], inplace=True)

# === 9. Exportar para Excel ===
df_sonia_emanu_sem_nf_recent.to_excel('emauel_sonia2.xlsx', index=False)

In [ ]:
# Caminho do relatório
path_relatorio = r"G:\Meu Drive\MASQUERADE\FINANCEIRO\RELATORIOS\CLIENTES-NOVOS\clientes_novos_região_08-2024_06-2025.xlsx"
workbook = xlsxwriter.Workbook(path_relatorio)
worksheet = workbook.add_worksheet('Relatório')

# Linha inicial
linha = 0

# Estilos
titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bold': True,
    'bg_color': "#72AC76", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'align': 'center', 'valign': 'vcenter'
})

# Cabeçalho principal
worksheet.merge_range("A1:H1", "RELATÓRIO DE CLIENTES NOVOS POR REGIÃO - 2024 - 2025", titulo)
linha += 2

# ---- Preparação dos dados

# Pivot
df_pivot = df_novos_raw.pivot_table(index='Periodo', columns='Região', aggfunc='size', fill_value=0)

# Garante todas as regiões
regioes = ['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul']
for reg in regioes:
    if reg not in df_pivot.columns:
        df_pivot[reg] = 0

df_pivot = df_pivot[regioes]
df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot.reset_index(inplace=True)

# Filtra por ano
df_2024 = df_pivot[df_pivot['Periodo'].str.startswith('2024')]
df_2025 = df_pivot[df_pivot['Periodo'].str.startswith('2025')]

headers = ['Período'] + regioes + ['Total']

# Função para escrever DataFrame
def escrever_df(worksheet, df, linha, format_info):
    for _, row in df.iterrows():
        for col, colname in enumerate(df.columns):
            worksheet.write(linha, col, row[colname], format_info)
        linha += 1
    return linha

# ---------- BLOCO 2024 ----------
worksheet.write(linha, 0, "2024", titulo)
linha += 1

for col, header in enumerate(headers):
    worksheet.write(linha, col, header, cols)
linha += 1

linha = escrever_df(worksheet, df_2024, linha, infos)

totais_2024 = df_2024[regioes + ['Total']].sum()
worksheet.write(linha, 0, 'Total 2024', cols)
for col, val in enumerate(totais_2024, start=1):
    worksheet.write(linha, col, val, infos)
linha += 2

# ---------- BLOCO 2025 ----------
worksheet.write(linha, 0, "2025", titulo)
linha += 1

for col, header in enumerate(headers):
    worksheet.write(linha, col, header, cols)
linha += 1

linha = escrever_df(worksheet, df_2025, linha, infos)

totais_2025 = df_2025[regioes + ['Total']].sum()
worksheet.write(linha, 0, 'Total 2025', cols)
for col, val in enumerate(totais_2025, start=1):
    worksheet.write(linha, col, val, infos)

# Finaliza o arquivo
workbook.close()
